In [ ]:
import airbnb
import pandas as pd
import datetime as dt
import credentials
from slackclient import SlackClient

In [ ]:
api = airbnb.Api(credentials.AIRBNB_EMAIL, credentials.AIRBNB_PASSWORD)
## this fails sometimes if you havent logged into the website on the same machine recently

In [ ]:
api.get_profile()

In [ ]:
#api doesn't seem able to return these anymore
listings=(17343208,
          17767863,
          20287325,
          31146290,
          26572144,
          17433075,
          17696905,
          23004935,
          17767353,
          21026079,
          21449226,
          28992621,
          29992190,
          17944621,
          26571969,
          21026964,
          21026891,
          17731097,
          17344333,
          27710210,
          27713331,
          27381476,
          24471037,
          24474059,
          24474631,
          24474830,
          18349074,
          30805198,
          30805783,
          30776488,
          21555212,
          22310307,
          18794629)

In [ ]:
def getReviewsFromListing(listing):
    offset=100
    df=pd.DataFrame.from_dict(api.get_reviews(listing, limit=100)['reviews'])
    dfTotal=df
    while len(df.index)==100:
        df=pd.DataFrame.from_dict(api.get_reviews(listing, limit=100,offset=offset)['reviews'])
        dfTotal=dfTotal.append(df)
        offset=offset+100
    return dfTotal

In [ ]:
##pull full reviews for all listings
##appends into big dataframe
def makeReviewDF(listings):
    print("Getting reviews from listing " + str(listings[0]))
    reviewsdf=getReviewsFromListing(listings[0])
    for listing in listings:
        if not(listing==listings[0]):
            print("Getting reviews from listing " + str(listing))
            reviewsdf=reviewsdf.append(getReviewsFromListing(listing))
    ##convert to datetime
#    reviewsdf['created_at']=pd.to_datetime(reviewsdf['created_at'])
    return reviewsdf

In [ ]:
reviewsdf=makeReviewDF(listings)

In [ ]:
print(reviewsdf.sort_values(by=['created_at'], ascending=False))

In [ ]:
def fixDate(date):
    return date.tz_localize(None)

def filterNewReviews(reviewsDF):
    today=dt.datetime.today()
    start=pd.Timestamp(today)+pd.Timedelta('-6 days')
    end=pd.Timestamp(today)+pd.Timedelta('-1 days')
    reviewsdf['created_at']=pd.to_datetime(reviewsdf['created_at']).apply(fixDate)
    df=reviewsdf[reviewsdf['created_at']>=start]
    df2=df[df['created_at']<end]
    return df2

In [ ]:
reviewsToSend=filterNewReviews(reviewsdf)
print(reviewsToSend)

In [ ]:
def formatReviews(reviewsdf):
    reviewList=[]
    size=len(reviewsdf.index)
    
    for i in range(0,len(reviewsdf.index)):
        name=reviewsToSend.iloc[i]['author']['smart_name']
        rating=reviewsToSend.iloc[i]['rating']
        review=reviewsToSend.iloc[i]['comments']
        date=reviewsToSend.iloc[i]['created_at'].strftime('%B %d, %Y, %r')
        room=reviewsToSend.iloc[i]['listing']['name']
        if ("Dave" in review) or ("dave" in review) or ("David" in review) or ("david" in review):
            pizza="\n"+":pizza_dance:"*10
        else:
            pizza=""
        output="\n"+ratingToEmoji(rating)+pizza+"\n\n Review for *" + room + "* by *" +name+ \
            "* on " + date + "\n\n*Rating*: "+":star:"*rating+"\n\n"+review+"\n\n"+ ratingToEmoji(rating)
        reviewList.append(output)
    return reviewList

In [ ]:
reviewsText=formatReviews(reviewsToSend)

In [ ]:
def postReviewsToSlack(reviewsList):
    # connect to slack and post the reviews with a delay
    token = credentials.SLACK_TOKEN      # found at https://api.slack.com/web#authentication
    sc = SlackClient(token)
    for review in reviewsList:
        sc.api_call(
            "chat.postMessage",
            channel='rv_hotel_reviews',
            text=review,
            icon_emoji=":love_hotel:"
        ) 

In [ ]:
postReviewsToSlack(reviewsText)